###In this notebook, we gonna try to leverage the result of an Azure Media Service Job to perform sentiment analysis on a podcast. 
The podcast is about a football French team (PSG).
Ideally we want to assess if the podcats is mostly positive or negative about PSG.

Type |information
--- | --- 
URL podcast used|https://audiormc.streamakaci.com/channel386/20210322_637520413273093658_seq_RMCAU_up_2021_3_22_20_19.mp3
Podcast Language|French
Input| Json file created by Azure Media Service
Azure Media Service references|https://docs.microsoft.com/en-us/azure/media-services/latest/analyze-video-audio-files-concept<br>https://docs.microsoft.com/en-us/azure/media-services/latest/transform-jobs-concept

###Step 1: Set up access to the storage

Using Service principal is preferred but it is not our focus in this notebook
<br>https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/adls-gen2/azure-datalake-gen2-sp-access

In [0]:
spark.conf.set(
   "fs.azure.account.key.lbdemomediaservices.blob.core.windows.net","{your storage account key}")

In [0]:
from pyspark.sql.functions import col, explode

###Step 2 Read the file created by Azure Media Service and apply some transformations

Azure Media Service after transforming the media creates different json files for different purposes
<br>We are only interested in the insights.json file (https://docs.microsoft.com/en-us/azure/media-services/latest/analyze-video-audio-files-concept#insightsjson-elements)

In [0]:
dfPodcast = spark.read.json("wasbs://{your storage account}@{your container}.blob.core.windows.net/insights.json")

In [0]:
display(dfPodcast)

duration keywords language languages observedPeople sentiments sourceLanguage sourceLanguages speakers statistics transcript version 0:15:09.504 List(List(0.9975, 1, List(List(0:00:29.49, 0:00:28.43, 0:00:29.49, 0:00:28.43), List(0:00:35.88, 0:00:32.42, 0:00:35.88, 0:00:32.42), List(0:01:01.684, 0:00:59.6, 0:01:01.684, 0:00:59.6)), fr-FR, ah oui), List(0.9975, 2, List(List(0:00:39.34, 0:00:35.88, 0:00:39.34, 0:00:35.88), List(0:06:39.59, 0:06:34.95, 0:06:39.59, 0:06:34.95), List(0:10:06.55, 0:10:01.66, 0:10:06.55, 0:10:01.66)), fr-FR, ps j'), List(0.9975, 3, List(List(0:01:22.06, 0:01:17.422, 0:01:22.06, 0:01:17.422), List(0:06:22.574, 0:06:21.638, 0:06:22.574, 0:06:21.638)), fr-FR, bon c'), List(0.9975, 4, List(List(0:02:36.21, 0:02:32.82, 0:02:36.21, 0:02:32.82), List(0:05:27.04, 0:05:24.96, 0:05:27.04, 0:05:24.96), List(0:10:00.12, 0:09:55.27, 0:10:00.12, 0:09:55.27), List(0:13:54.15, 0:13:46.17, 0:13:54.15, 0:13:46.17)), fr-FR, a beaucoup), List(0.9975, 5, List(List(0:10:06.55, 0:10:01.66, 0:10:06.55, 0:10:01.66)), fr-FR, passé face), List(0.9975, 6, List(List(0:11:53.35, 0:11:50.17, 0:11:53.35, 0:11:50.17), List(0:12:51.558, 0:12:47.98, 0:12:51.558, 0:12:47.98)), fr-FR, va falloir), List(0.9966, 7, List(List(0:05:19.88, 0:05:18.49, 0:05:19.88, 0:05:18.49), List(0:05:22.16, 0:05:19.88, 0:05:22.16, 0:05:19.88)), fr-FR, moins bon), List(0.9966, 8, List(List(0:03:19.6, 0:03:17.71, 0:03:19.6, 0:03:17.71), List(0:03:55.59, 0:03:49.781, 0:03:55.59, 0:03:49.781), List(0:04:03.04, 0:03:57.38, 0:04:03.04, 0:03:57.38), List(0:04:22.57, 0:04:19.26, 0:04:22.57, 0:04:19.26), List(0:04:41.58, 0:04:38.49, 0:04:41.58, 0:04:38.49), List(0:04:50.01, 0:04:41.58, 0:04:50.01, 0:04:41.58), List(0:04:56.75, 0:04:52.82, 0:04:56.75, 0:04:52.82), List(0:08:13.63, 0:08:09.3, 0:08:13.63, 0:08:09.3), List(0:08:30.38, 0:08:26.08, 0:08:30.38, 0:08:26.08), List(0:09:05.99, 0:09:03.39, 0:09:05.99, 0:09:03.39), List(0:10:11.17, 0:10:08.6, 0:10:11.17, 0:10:08.6), List(0:12:29.85, 0:12:26.45, 0:12:29.85, 0:12:26.45), List(0:13:34.47, 0:13:29.83, 0:13:34.47, 0:13:29.83)), fr-FR, paris saint), List(0.9961, 9, List(List(0:00:41.73, 0:00:39.34, 0:00:41.73, 0:00:39.34), List(0:07:09.02, 0:07:05.66, 0:07:09.02, 0:07:05.66), List(0:07:11.92, 0:07:09.78, 0:07:11.92, 0:07:09.78)), fr-FR, plus dur), List(0.9954, 10, List(List(0:11:59.35, 0:11:53.35, 0:11:59.35, 0:11:53.35)), fr-FR, carton jaune), List(0.9954, 11, List(List(0:14:47.74, 0:14:41.69, 0:14:47.74, 0:14:41.69)), fr-FR, rapport à), List(0.9949, 12, List(List(0:06:19.35, 0:06:16.49, 0:06:19.35, 0:06:16.49), List(0:15:01.85, 0:14:55.8, 0:15:01.85, 0:14:55.8)), fr-FR, semaine prochaine), List(0.9944, 13, List(List(0:03:17.71, 0:03:13.32, 0:03:17.71, 0:03:13.32), List(0:09:45.52, 0:09:42.97, 0:09:45.52, 0:09:42.97)), fr-FR, fallait faire), List(0.9932, 14, List(List(0:03:57.38, 0:03:55.59, 0:03:57.38, 0:03:55.59), List(0:11:42.05, 0:11:37.46, 0:11:42.05, 0:11:37.46), List(0:11:59.35, 0:11:53.35, 0:11:59.35, 0:11:53.35)), fr-FR, match aller), List(0.9932, 15, List(List(0:05:12.95, 0:05:08.67, 0:05:12.95, 0:05:08.67), List(0:11:59.35, 0:11:53.35, 0:11:59.35, 0:11:53.35)), fr-FR, match retour), List(0.9916, 16, List(List(0:04:31.5, 0:04:24.55, 0:04:31.5, 0:04:24.55), List(0:06:34.95, 0:06:32.61, 0:06:34.95, 0:06:32.61)), fr-FR, l'autre côté), List(0.9903, 17, List(List(0:10:00.12, 0:09:55.27, 0:10:00.12, 0:09:55.27), List(0:13:00.87, 0:12:58.27, 0:13:00.87, 0:12:58.27), List(0:14:35.64, 0:14:28.39, 0:14:35.64, 0:14:28.39)), fr-FR, contre lyon), List(0.9887, 18, List(List(0:01:10.74, 0:01:08.572, 0:01:10.74, 0:01:08.572), List(0:01:29.23, 0:01:26.75, 0:01:29.23, 0:01:26.75)), fr-FR, première fois), List(0.9781, 19, List(List(0:05:35.26, 0:05:30.97, 0:05:35.26, 0:05:30.97), List(0:05:37.65, 0:05:36.62, 0:05:37.65, 0:05:36.62)), fr-FR, vrai problème), List(0.8814, 20, List(List(0:00:13.45, 0:00:00, 0:00:13.45, 0:00:00), List(0:00:52.9, 0:00:49.71, 0:00:52.9, 0:00:49.71), List(0:03:19.6, 0:03:17.71, 0:03:19.6, 0:03:17.7

Our objective is to focus on sentiments analysis per period. It is a nested structure. We will extract from the json file only the time period, the sentiments and the text.

**First transformation: Isolate the text per time period**

In [0]:
dfTranscript = dfPodcast.select(explode(col("transcript")).alias("extract")).select(col("extract.confidence").alias("confidence"),explode(col("extract.instances")).alias("instances"),col("extract.text").alias("text")).select("confidence","instances.adjustedStart","instances.adjustedEnd","text")

In [0]:
display(dfTranscript)

confidence,adjustedStart,adjustedEnd,text
0.4519,0:00:00,0:00:13.45,RMC. Le podcast Gilbert brisbois. Chers supporters de Nice 15 kebano et de Yvon Le Roux Bienvenue dans le podcast after Paris.
0.4355,0:00:13.45,0:00:17.12,"15 minutes de débats consacrés à l'actu du PSG avec aujourd'hui coach Courbis,"
0.4355,0:00:17.12,0:00:18.53,"salut Roland, Salut les amis,"
0.4355,0:00:18.53,0:00:20.645,"salut à tous et avec Loïc tanzi,"
0.4355,0:00:20.707,0:00:22.494,"qui suit le PS, j'ai toute l'année,"
0.4355,0:00:22.54,0:00:23.759,"salut Loïc. Salut à tous,"
0.4355,0:00:23.822,0:00:26.71,"kebano, la Sacem, Roland qui était pas là sur kebano."
0.5151,0:00:28.43,0:00:29.49,"Ah oui, ça, c'est"
0.5151,0:00:29.49,0:00:32.42,"moi qui n'est pas là pour le rouge champion d'Europe 84,"
0.5151,0:00:32.42,0:00:35.88,Oh bah oui. Au programme l'évaluation après le match à Lyon bien sûr.


**Second transformation: Isolate the sentiment per time period**

In [0]:
dfSentiments = dfPodcast.select(explode(col("sentiments")).alias("extract")).select(col("extract.sentimentType").alias("sentiment"),explode(col("extract.instances")).alias("windows_time")).select("sentiment","windows_time.adjustedStart","windows_time.adjustedEnd")

In [0]:
display(dfSentiments)

sentiment,adjustedStart,adjustedEnd
Negative,0:01:26.75,0:01:29.23
Negative,0:03:29.33,0:03:32.526
Negative,0:07:09.78,0:07:11.92
Negative,0:08:32.7,0:08:36.322
Negative,0:09:32.76,0:09:34.818
Negative,0:09:47.61,0:09:49.7
Negative,0:10:31.1,0:10:33.67
Negative,0:11:29.989,0:11:32.21
Negative,0:13:23.123,0:13:24.27
Neutral,0:00:00,0:00:01.38


At first glance it looks like the podcast is mostly neutral and twice postive than negative

We want to be able to check the transcript and sentiment analysis per time period

In [0]:
dfSentimentAnalysis = dfTranscript.join(dfSentiments,how="left", on=[dfTranscript.adjustedStart == dfSentiments.adjustedStart,dfTranscript.adjustedEnd == dfSentiments.adjustedEnd])

###Step 3 Explore the results

The results of the transcript is pretty impressive

In [0]:
display(dfSentimentAnalysis)

confidence,adjustedStart,adjustedEnd,text,sentiment,adjustedStart,adjustedEnd
0.4519,0:00:00,0:00:13.45,RMC. Le podcast Gilbert brisbois. Chers supporters de Nice 15 kebano et de Yvon Le Roux Bienvenue dans le podcast after Paris.,null,null,null
0.4355,0:00:13.45,0:00:17.12,"15 minutes de débats consacrés à l'actu du PSG avec aujourd'hui coach Courbis,",null,null,null
0.4355,0:00:17.12,0:00:18.53,"salut Roland, Salut les amis,",Positive,0:00:17.12,0:00:18.53
0.4355,0:00:18.53,0:00:20.645,"salut à tous et avec Loïc tanzi,",null,null,null
0.4355,0:00:20.707,0:00:22.494,"qui suit le PS, j'ai toute l'année,",null,null,null
0.4355,0:00:22.54,0:00:23.759,"salut Loïc. Salut à tous,",null,null,null
0.4355,0:00:23.822,0:00:26.71,"kebano, la Sacem, Roland qui était pas là sur kebano.",null,null,null
0.5151,0:00:28.43,0:00:29.49,"Ah oui, ça, c'est",null,null,null
0.5151,0:00:29.49,0:00:32.42,"moi qui n'est pas là pour le rouge champion d'Europe 84,",null,null,null
0.5151,0:00:32.42,0:00:35.88,Oh bah oui. Au programme l'évaluation après le match à Lyon bien sûr.,null,null,null


It looks like the approach is a not the best one. During the podcast, there are a few moments where the presenters don't speak about PSG. A few examples:
* At the beginning, it is mostly greetings which are considered positive but not related to PSG
* A couple of times, the presenters made digressions and so the sentiment analysis is not related to PSG.

In this specific case, couple of things could be used to have a better result:
* More transformation to isolate only the parts of the podcast where the presenter speak about PSG
* Maybe use an NLP library that could offer more options like isolate the different topics of the podcast

We could have apply more transformation to make sure that each text is linked to a sentiment analysis. However as explained above, it is better to focus on completing/modifying the approach before doing so.
<br>However for clarity, we want to ignore texts where the sentiment is null

In [0]:
display(dfSentimentAnalysis.filter(col("sentiment").isNotNull()))

confidence,adjustedStart,adjustedEnd,text,sentiment,adjustedStart,adjustedEnd
0.4355,0:00:17.12,0:00:18.53,"salut Roland, Salut les amis,",Positive,0:00:17.12,0:00:18.53
0.4231,0:01:26.75,0:01:29.23,c'est la première fois qu'on le voit pas voilà.,Negative,0:01:26.75,0:01:29.23
0.5515,0:02:26.23,0:02:27.45,je l'ai trouvé très bon.,Positive,0:02:26.23,0:02:27.45
0.4935,0:03:55.59,0:03:57.38,"Contrairement au match aller, bien sûr.",Positive,0:03:55.59,0:03:57.38
0.5343,0:06:09.21,0:06:11.81,"Pourquoi pas, mais c'est vrai que ça serait le point,",Positive,0:06:09.21,0:06:11.81
0.7798,0:07:09.78,0:07:11.92,Mais le plus dur en fait quand,Negative,0:07:09.78,0:07:11.92
0.6692,0:07:32.26,0:07:36.32,"disons que bon peut être je suis trop optimiste moi,",Positive,0:07:32.26,0:07:36.32
0.562,0:09:47.61,0:09:49.7,"ils ont perdu, mais là dans dans l'état d'esprit,",Negative,0:09:47.61,0:09:49.7
0.8096,0:10:31.1,0:10:33.67,Je. Je souhaite à personne de de se blesser.,Negative,0:10:31.1,0:10:33.67
0.1861,0:15:01.85,0:15:03.36,"Merci Loïc. Merci Roland. Salut,",Positive,0:15:01.85,0:15:03.36
